# D - Global Variables in Stored Procedures

There are various global variables in the Database Server that we can make use of in our scripts. Three commonly used ones include:

- `@@IDENTITY` - Hold the value for an `IDENTITY` column generated during the last SQL `INSERT` statement for a table with an Identity column.
- `@@ROWCOUNT` - An integer identifying how many rows were affected by the last SQL operation.
- `@@ERROR` - An integer for the error code that was generated by the last SQL operation. A value of zero for `@@ERROR` means that there was no error in the operation.



Let's examine the following problem that has us making use of the `@@IDENTITY` variable.

> 1. Create a stored procedure called **AddPosition** that will accept a Position Description (varchar 50). Return the primary key value that was database-generated as a result of your Insert statement. Also, ensure that the supplied description is not NULL and that it is at least 5 characters long. Make sure that you do not allow a duplicate position name.

We should approach these problems with a set of standard questions for our analysis.

- What information needs to be supplied to our stored procedure? (These items will become parameters for our stored procedure.)
  - The problem statement may explicitly identify information passed to the sproc. Alternativly, there may be some implied pieces of information that we will need for our sproc to work.
- What information will be returned from calling our stored procedure?
- What table(s) will we have to work with in our stored procedure?

With these basic questions answered, we start to frame out our stored procedure.

In [ ]:
CREATE OR ALTER PROCEDURE AddPosition
    -- Parameters here
    @Description     varchar(50)
AS
    -- Body of procedure here
RETURN

The information coming in is the *description* that we need to put into the `Position` table, and the information that our stored procedure needs to return is the Identity value generated by the database for the `PositionID` column.

The heart of our sproc's instructions will be an `INSERT` statement.

In [ ]:
INSERT INTO Position(PositionDescription)
VALUES (@Description)
-- As soon as this statement executes, the global variable @@IDENTITY 
-- will have the same value that was inserted into the PositionID column
SELECT @@IDENTITY AS 'NewPositionID'

We also have to do some validation, specifically checking that the supplied parameter is not `NULL` and that it is at least 5 characters long. We also need to make sure that the description doesn't already exist in the table.

The pattern I recommend for validation is to ask "Is there a problem with *X*, else...", where you address each problem in turn before attempting the main task of your sproc.

In [1]:
CREATE OR ALTER PROCEDURE AddPosition
    -- Parameters here
    @Description    varchar(50) -- Max of 50 characters
AS
    -- Body of procedure here
    IF @Description IS NULL
    BEGIN -- {
        RAISERROR('Description is required', 16, 1) -- Throw an exception
    END   -- }
    ELSE
    BEGIN -- {
        IF LEN(@Description) < 5
        BEGIN -- {
            RAISERROR('Description must be between 5 and 50 characters', 16, 1)
        END   -- }
        ELSE
        BEGIN -- {
            IF EXISTS(SELECT * FROM Position WHERE PositionDescription = @Description)
            BEGIN -- {
                RAISERROR('Duplicate positions are not allowed', 16, 1)
            END   -- }
            ELSE
            BEGIN -- { -- This BEGIN/END is needed, because of two SQL statements
                INSERT INTO Position(PositionDescription)
                VALUES (@Description)
                -- Send back the database-generated primary key
                SELECT @@IDENTITY AS 'NewPositionID' -- This is a global variable
            END   -- }
        END   -- }
    END   -- }
RETURN

Commands completed successfully.

Total execution time: 00:00:00.020

Once our stored procedure is in place, we can test our stored procedure with the following call.

In [3]:
EXEC AddPosition 'The Boss' -- this is a test of the "happy path"

: Msg 50000, Level 16, State 1, Procedure AddPosition, Line 20
Duplicate positions are not allowed

Total execution time: 00:00:00.007

In [4]:
EXEC AddPosition NULL

: Msg 50000, Level 16, State 1, Procedure AddPosition, Line 8
Description is required

Total execution time: 00:00:00

In [5]:
EXEC AddPosition 'Me'

: Msg 50000, Level 16, State 1, Procedure AddPosition, Line 14
Description must be between 5 and 50 characters

Total execution time: 00:00:00

In [6]:
EXEC AddPosition 'The Boss' -- this should result in an error as a duplicate

: Msg 50000, Level 16, State 1, Procedure AddPosition, Line 20
Duplicate positions are not allowed

Total execution time: 00:00:00.001

What will happen if I try to run the stored procedure by passing in a string value that is too big?

In [7]:
EXEC AddPosition 'The Boss of everything and everyone, everywhere and all the time, both past present and future, without any possible exception. Unless, of course, I''m not...'

(1 row affected)

(1 row affected)

Total execution time: 00:00:00.010

NewPositionID
11


In [8]:
SELECT * FROM Position

(9 rows affected)

Total execution time: 00:00:00.014

PositionID,PositionDescription
1,Dean
2,Program Chair
3,Assistant Program Chair
4,Instructor
5,Office Administrator
6,Technical Support Staff
7,Assistant Dean
10,The Boss
11,"The Boss of everything and everyone, everywhere an"
